In [187]:
import pandas as pd
import numpy as np

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import xgboost as xgb
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import  pyLDAvis.sklearn

from tqdm import tqdm
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string

# !python3 -m spacy download en_core_web_lg --user

In [2]:
nlp = spacy.load('en_core_web_lg')

In [18]:
data = pd.read_csv("./winemag-data_first150k.csv")

data = data[data.duplicated('description', keep=False)]
print("Length of dataframe after duplicates are removed:", len(data))

data.dropna(subset=['description', 'points'])
print("Length of dataframe after NaNs are removed:", len(data))

Length of dataframe after duplicates are removed: 92393
Length of dataframe after NaNs are removed: 92393


In [136]:
labels = data['points']
descriptions = data['description']

In [20]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [21]:
# Parser for reviews
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [22]:
tqdm.pandas()
data["processed_description"] = data["description"].progress_apply(spacy_tokenizer)

100%|██████████| 92393/92393 [02:07<00:00, 722.60it/s]


In [23]:
vectorizer = TfidfVectorizer()
data_vectorized = vectorizer.fit_transform(data["processed_description"])

In [37]:
lda = LatentDirichletAllocation(n_components=10, doc_topic_prior=.01, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [44]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

/home/prdx/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6     -153.850037 -276.474365       1        1  23.361877
5      122.237991   11.392784       2        1  20.579814
4      149.747864  230.453049       3        1  19.822844
8     -284.986053  299.651794       4        1  17.772866
2     -118.155678  -52.727093       5        1   6.973384
3       80.361496 -205.382507       6        1   2.957162
9      -50.797279  370.776093       7        1   2.298698
0     -354.363159 -136.131897       8        1   2.279750
1      -86.362930  147.008591       9        1   2.129803
7     -326.700775   82.877380      10        1   1.823801, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
3272   Default  2436.000000        cherry  2436.000000  30.0000  30.0000
17226  Default   616.000000      vineyard   616.000000  29.0000  29.0000
11968  Default   875.000000         pinot   875.000000  28.0000  28.0000
11402  Default   539.000000          pair   539.000000  27.0000  27.0000
3723   Default   790.000000         colon   790.000000  26.0000  26.0000
15731  Default  2090.000000        tannin  2090.000000  25.0000  25.0000
11628  Default   972.000000         peach   972.000000  24.0000  24.0000
3463   Default  1076.000000        citrus  1076.000000  23.0000  23.0000
14357  Default   449.000000         shows   449.000000  22.0000  22.0000
11412  Default  1652.000000        palate  1652.000000  21.0000  21.0000
1093   Default  1144.000000         apple  1144.000000  20.0000  20.0000
7994   Default   575.000000      imported   575.000000  19.0000  19.0000
16967  Default   275.000000        valley   275.000000  18.0000  18.0000
11954  Default   812.000000     pineapple   812.000000  17.0000  17.0000
14428  Default   502.000000         silky   502.000000  16.0000  16.0000
1213   Default  1940.000000         aroma  1940.000000  15.0000  15.0000
11637  Default   826.000000          pear   826.000000  14.0000  14.0000
10801  Default   471.000000          noir   471.000000  13.0000  13.0000
644    Default  1094.000000           age  1094.000000  12.0000  12.0000
1956   Default  1681.000000         black  1681.000000  11.0000  11.0000
4310   Default  1301.000000         crisp  1301.000000  10.0000  10.0000
6324   Default  3571.000000        flavor  3571.000000   9.0000   9.0000
17855  Default   186.000000     zinfandel   186.000000   8.0000   8.0000
9122   Default   658.000000          lime   658.000000   7.0000   7.0000
3191   Default   709.000000    chardonnay   709.000000   6.0000   6.0000
17627  Default   322.000000        winery   322.000000   5.0000   5.0000
15286  Default   862.000000         style   862.000000   4.0000   4.0000
17765  Default  1282.000000          year  1282.000000   3.0000   3.0000
14618  Default   662.000000         smoky   662.000000   2.0000   2.0000
15379  Default   228.000000        sugary   228.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
15733  Topic10    62.292700       tannins    63.126245   3.9910  -4.7765
17457  Topic10    58.542664        wealth    59.376110   3.9901  -4.8386
4794   Topic10    50.749914     designate    51.583721   3.9880  -4.9815
4376   Topic10    50.727230         crust    51.560954   3.9879  -4.9819
12054  Topic10    48.703973       playful    49.537476   3.9873  -5.0226
13853  Topic10    44.238528       sausage    45.071801   3.9856  -5.1188
12353  Topic10    41.289508      presence    42.122884   3.9843  -5.1878
11173  Topic10    37.993863   organically    38.827378   3.9825  -5.2710
7195   Topic10    37.299896        graham    38.132992   3.9822  -5.2894
15882  Topic10    36.295234          tend    37.128673   3.9815  -5.3167
13332  Topic10    36.246316        ripely    37.079581   3.9815  -5.3180
2554   Topic10    35.213834    burgundian    36.0

In [92]:
X = pd.DataFrame(data_lda)

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

------------------------------------------------------------------------------------

In [184]:
parameters = {'n_estimators': [500, 700, 1000], 'max_depth': [None, 10, 100]}

rf_cv = GridSearchCV(RandomForestRegressor(), parameters, cv=10, scoring="r2",
                   n_jobs=8, verbose=1)
rf_cv.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [186]:
pred = rfc.predict(X_test)
mean_squared_error(y_test, pred)

1.3122903462459368

--------------------------------------------------------------------------------------------------

In [146]:
parameters = {'max_depth': [None, 10, 100], 'n_estimators': [500, 700, 1000]}

xgb_cv = GridSearchCV(xgb.XGBRegressor(), parameters, cv=10, scoring="r2", 
                   n_jobs=8, verbose=1)
xgb_cv.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=100, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

---------------------------------------------------------------------------------

In [165]:
parameters = {alpha=[1, 0.1, 0.01, 0.001]}
ridge_cv = GridSearchCV(Ridge(), parameters, cv=10, scoring="r2", 
                   n_jobs=8, verbose=1)
ridge_cv.fit(X_train, y_train)

Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [166]:
from sklearn.metrics import mean_squared_error

In [167]:
mean_squared_error(y_test, lr.predict(X_test))

8.482839988033524